In [5]:
# 15마리 처치시 아이템 생성 획득시 무기 변환 
# 거리와 몬스터 잡을때마다 점수
# 보스몬스터 (몬스터 20마리 사냥시) # boss false시로 설정 
# 적 20마리 처치시 적 속도 증가
# 운석 생성 (몬스터를 3마리 놓쳤을 때)
# 속도는 초속 1M씩 움직인다

import pygame
import sys
from time import sleep
import random
import time
import datetime

padWidth=480
padHeight=800
enemyImage=['enemy1.png','enemy2.png','enemy3.png','enemy4.png','enemy5.png','enemy6.png'
          ,'enemy7.png','enemy8.png']
bossImage=['boss1.png','boss2.png','boss3.png']

def writeScore(count):
    global gamePad
    font=pygame.font.Font('NanumGothicCoding.ttf',20)
    text=font.render('점수:'+str(count),True,(255,255,255))
    gamePad.blit(text,(10,0))

def writeDistance(count):
    global gamePad
    font=pygame.font.Font('NanumGothicCoding.ttf',20)
    text=font.render('거리:'+str(count)+'M',True,(255,255,255))
    gamePad.blit(text,(360,0))

def writeMessage(text):
    global gamePad
    font=pygame.font.Font('NanumGothicCoding.ttf',80)
    text=font.render(text,True,(255,0,0))
    textpos=text.get_rect()
    textpos.center=(padWidth/2,padHeight/2)
    gamePad.blit(text,textpos)
    pygame.display.update()
    sleep(2)
    runGame()

    
def crash():
    global gamePad
    writeMessage('게임 오버!')
    pygame.quit()
    
def gameClear():
    global gamePad
    writeMessage('게임 승리!')
    pygame.quit()

def drawObject(obj,x,y):
    global gamePad
    gamePad.blit(obj,(x,y))

def initGame():
    global gamePad, clock, background, fighter, missile, explosion, item
    pygame.init()
    gamePad=pygame.display.set_mode((padWidth,padHeight))
    pygame.display.set_caption('Dragon Flight')#게임 이름
    background=pygame.image.load('background.png') #배경 그림
    fighter=pygame.image.load('character.png')
    missile = pygame.image.load('weapon.png')
    explosion = pygame.image.load('exp.png')
    item = pygame.image.load('item.png')
    clock = pygame.time.Clock()
    
def runGame():
    global gamepad, clock, background, fighter, missile, explosion, item
    
    start = time.time()
    #무기 좌표
    missileXY=[]
    
    #장애물 랜덤 생성
    enemy=pygame.image.load(random.choice(enemyImage))
    enemySize=enemy.get_rect().size
    enemyWidth=enemySize[0]
    enemyHeight=enemySize[1]
    
    #적 위치 초기화
    enemyX=random.randrange(0,padWidth-enemyWidth)
    enemyY=0
    enemySpeed=2
    
    #운석 생성
    rock=pygame.image.load('rock.png')
    rockSize=rock.get_rect().size
    rockWidth=rockSize[0]
    rockHeight=rockSize[1]
    
    #운석 위치 초기화
    rockX=random.randrange(0,padWidth-rockWidth)
    rockY=0
    rockSpeed=6
    
    
    #보스 생성
    boss=pygame.image.load(random.choice(bossImage))
    bossSize=boss.get_rect().size
    bossWidth=bossSize[0]
    bossHeight=bossSize[1]
    
    #보스 위치
    bossX=60
    bossY=0
    
    #캐릭터
    fighterSize=fighter.get_rect().size
    fighterWidth=fighterSize[0]
    fighterHeight=fighterSize[1]
    
    #아이템
    itemSize=item.get_rect().size
    itemWidth=itemSize[0]
    itemHeight=itemSize[1]
    
    itemX=random.randrange(0,padWidth-rockWidth)
    itemY=0
    itemSpeed=6
    
    
    print(rockWidth,' ',enemyWidth, fighterWidth)

    x=padWidth*0.45
    y=padHeight*0.9
    fighterX=0
    
    #미사일이 운석에 맞았을 경우 True
    isShot=False
    shotCount=0
    enemyPassed=0
    bossShot=False
    bossShotCount=0
    
    bossExist=False # 보스 유무
    onGame=False
    while not onGame:
        for event in pygame.event.get():
            if event.type in [pygame.QUIT]: # 게임 종료
                pygame.quit()
                sys.exit()
                
            if event.type in [pygame.KEYDOWN]: # 캐릭터 왼쪽 이동
                if event.key==pygame.K_LEFT:
                    fighterX-=5
                elif event.key==pygame.K_RIGHT: #캐릭터 오른쪽 이동
                    fighterX+=5
                elif event.key==pygame.K_SPACE: # 무기 발사
                    missileX=x+fighterWidth/2
                    missileY=y-fighterHeight
                    missileXY.append([missileX,missileY])
                
            if event.type in [pygame.KEYUP]: # 캐릭터 멈춤
                if event.key==pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    fighterX=0
                    
        x+=fighterX # 캐릭터 위치 재조정
        if x<0:
            x=0
        elif x> padWidth-fighterWidth:
            x=padWidth-fighterWidth
        
        #게임 오버
        if y<enemyY+enemyHeight:
            if(enemyX>x and enemyX < x + fighterWidth) or (enemyX + enemyWidth > x and enemyX + enemyWidth < x + fighterWidth):            
                crash()
        if y<rockY+rockHeight:
            if (rockX>x and rockX < x + fighterWidth) or (rockX + rockWidth > x and rockX + rockWidth < x + fighterWidth):
                crash()
        if y<itemY+itemHeight:
            if (itemX>x and itemX < x + fighterWidth) or (itemX + rockWidth > x and itemX + itemWidth < x + fighterWidth):
                item = pygame.image.load('item2.png')
                missile = pygame.image.load('weapon2.png')
                
        
        drawObject(background,0,0)  
        drawObject(fighter,x,y)# 캐릭터 그리기
        
        
        
        if len(missileXY) !=0:
            for i, bxy in enumerate(missileXY):
                bxy[1]-=10
                missileXY[i][1]=bxy[1]
                
                #미사일이 보스 맞추었을때
                if bossExist:
                    if bxy[1]<bossWidth and bossExist:
                        if bxy[0]>bossY and bxy[0]< bossX+bossWidth:
                            missileXY.remove(bxy)                        
                            bossShot=True
                            bossShotCount+=1
                else: #미사일이 적 맞추었을때
                    if bxy[1]<enemyY and not bossExist:
                        if bxy[0]>enemyX and bxy[0] < enemyX+enemyWidth:
                            missileXY.remove(bxy)
                            isShot=True
                            shotCount+=10
                
                if bxy[1]<=0:
                    try:
                        missileXY.remove(bxy)
                    except:
                        pass
                    
        if len(missileXY)!=0:
            for bx,by in missileXY:
                drawObject(missile,bx,by)#미사일그리기
        writeScore(shotCount)
        enemyY+=enemySpeed#적 아래로 떨어지게
        
        if shotCount==30:
            itemY+=itemSpeed
            drawObject(item,itemX,itemY)
        
        if shotCount==250:
            enemySpeed=8
            
        #보스 생성, 적 50마리 처치시
        if shotCount==500:
            bossExist=True
            drawObject(boss,bossX,bossY)    
            
        #보스 삭제
        if bossShot and bossShotCount==10:
            drawObject(explosion,enemyX,enemyY)# 보스 폭발 그리기
            bossExist=False
            print(bossShotCount)            
            bossShotCount=0
            bossShot=False
            gameClear()

        
        if bossExist:
            rockSpeed=8
            rockY+=rockSpeed
            drawObject(rock,rockX,rockY)
            
        #적 3마리 놓쳤을때, 운석을 피한 경우
        if rockY>padHeight and enemyPassed>3:
            #새로운 운석
            rock=pygame.image.load('rock.png')
            rockSize=rock.get_rect().size
            rockWidth=rockSize[0]
            rockHeight=rockSize[1]
            rockX=random.randrange(0,padWidth-rockWidth)
            rockY=0   
        
        #보스가 있을 때, 운석을 피한 경우
        if rockY>padHeight and bossExist:
            #새로운 운석
            rock=pygame.image.load('rock.png')
            rockSize=rock.get_rect().size
            rockWidth=rockSize[0]
            rockHeight=rockSize[1]
            rockX=random.randrange(0,padWidth-rockWidth)
            rockY=0   
            
        #적 20마리 처치시 운석 생성
        if enemyPassed>3:
            rockY+=rockSpeed
            drawObject(rock,rockX,rockY)
        
            
        #적을 맞춘 경우
        if isShot and not bossExist:
            drawObject(explosion,enemyX,enemyY)# 적 폭발 그리기
            
            #새로운 적
            enemy=pygame.image.load(random.choice(enemyImage))
            enemySize=enemy.get_rect().size
            enemyWidth=enemySize[0]
            enemyHeight=enemySize[1]
            enemyX=random.randrange(0,padWidth-enemyWidth)
            enemyY=0
            isShot=False
        
        
        #적을 놓칠경우
        if enemyY>padHeight and not bossExist:
            #새로운 운석
            enemy=pygame.image.load(random.choice(enemyImage))
            enemySize=enemy.get_rect().size
            enemyWidth=enemySize[0]
            enemyHeight=enemySize[1]
            enemyX=random.randrange(0,padWidth-enemyWidth)
            enemyY=0
            enemyPassed+=1
        
        if not bossExist:
            drawObject(enemy,enemyX,enemyY)#적 그리기
        
        end=time.time()
        distance=end-start
        writeDistance(round(distance))
        

        pygame.display.update() # 게임화면 다시 그리기
        
        
        clock.tick(60) #게임화면 프레임수
    pygame.quit() # 게임종료


initGame()
runGame()

103   128 71
103   128 71


SystemExit: 